## Generate Questions using the Della-Inference API

In [33]:
import requests
import json
import re
import pandas as pd

In [25]:
url = "http://localhost:12257/v1/chat/completions"

headers={
    "Content-Type": "application/json",
    "Authorization": "token-abc123"
}

model_name = 'meta-llama/Meta-Llama-3.1-70B-Instruct'

In [77]:
def get_question_generation_prompt(justice_name, opening_statement):
    #     system_prompt = """Below is an opening statement from a Supreme Court case and a question asked during the argument. Your task is to determine whether the question logically follows from the opening statement alone.

    #     Repond with a only a single word. If the question makes sense based solely on the content of the opening statement, label it as "coherent." If additional context from other parts of the argument is needed for the question to make sense, label it as "incoherent."
    #     """
    #     user_prompt = f"""
    #     ### Opening Statement:
    #     {opening_statement}

    #     ### Questions:
    #     """
    system_prompt = """You are a legal expert trained to simulate the questioning style of Supreme Court justices during oral arguments. Below is an opening statement from a Supreme Court case and the name of a specific justice. Your task is to generate a list of questions that this justice is likely to ask in response to the opening statement. These questions should reflect the justice’s known priorities, jurisprudence, and typical questioning style, and they should be directly relevant to the arguments presented in the opening statement.

        ### Instructions:
        1. Carefully analyze the opening statement to identify key arguments, assumptions, or ambiguities.
        2. Generate questions that the specified justice might ask.
        3. Format each question with specific start and end tags for easier parsing:
           - Use `<START_Q>` at the beginning of each question.
           - Use `<END_Q>` at the end of each question.

        ### Example:
        Justice: Justice Oliver Wendell Holmes Jr.
        Opening Statement: Congress intended a preponderance of the evidence standard to apply to FLSA exemptions. Respondents argue for a clear and convincing evidence standard due to the importance of overtime rights. However, waivability of rights and standards of proof are unrelated.

        Generated Questions:
        <START_Q> Is there any explicit legislative language that supports your claim regarding the preponderance standard? <END_Q>
        <START_Q> How does the principle of statutory construction guide us in resolving ambiguities in this case? <END_Q>
        <START_Q> Would adopting a clear and convincing standard conflict with the broader purpose of the Fair Labor Standards Act? <END_Q>
        <START_Q> How do you reconcile your argument with precedents where the Court imposed heightened standards for "important" rights? <END_Q>

        ### Your Task:
        Justice: {justice_name}
        Opening Statement: {opening_statement}

        ### Output:
        A list of questions that {justice_name} is likely to ask:
        <START_Q> text of question one <END_Q>
        <START_Q> text of question two <END_Q>
        <START_Q> text of question three <END_Q>

    """
    
    user_prompt = f"""### Your Task:
        Justice: {justice_name}
        Opening Statement: {opening_statement}

        ### Output:
    """

    messages = [
            {
                "role": "system",
                "content": system_prompt,
            },
            {"role": "user", "content": user_prompt}
        ]
    return messages

In [96]:
def get_model_response(messages):

    payload = {
        "model": model_name,
        "messages": messages
    }

    response = requests.post(url, data=json.dumps(payload), headers=headers)
    return response

def parse_response(response):
    decoded = response.content.decode('utf-8')
    response_data = json.loads(decoded)
    content = response_data['choices'][0]['message']['content']
    
    questions = re.findall(r"<START_Q>(.*?)<END_Q>", content, re.DOTALL)
    cleaned_questions = [q.strip() for q in questions]

    return cleaned_questions

#### Read csv with dataframe and try call

In [ ]:
input_fp = '../datasets/2024_full_text_transcripts.csv'
df = pd.read_csv(input_fp)
df.head()

,Unnamed: 0,transcript_id,petitioner_opening_text,petitioner_full_text,respondent_opening_statement,respondent_full_text
0,0,2024.23-621-t01,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Anthony A. Yang</speaker><text> Mr. C...,<speaker>Anthony A. Yang</speaker><text> Mr. C...
1,1,2024.23-365 -t01,<speaker>Lisa S. Blatt</speaker><text> Thank y...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Easha Anand</speaker><text> Mr. Chief...,<speaker>Easha Anand</speaker><text> Mr. Chief...
2,2,2024.23-852-t01,<speaker>Elizabeth B. Prelogar</speaker><text>...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Peter A. Patterson</speaker><text> Mr...,<speaker>Peter A. Patterson</speaker><text> Mr...
3,3,2024.23-980-t01,<speaker>Kannon K. Shanmugam</speaker><text> T...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Kevin K. Russell</speaker><text> Mr. ...,<speaker>Kevin K. Russell</speaker><text> Mr. ...
4,4,2024.23-191 -t01,<speaker>Adam G. Unikowsky</speaker><text> Mr....,"<speaker>John G. Roberts, Jr.</speaker><text> ...","<speaker>Edmund G. LaCour, Jr.</speaker><text>...","<speaker>Edmund G. LaCour, Jr.</speaker><text>..."


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Unnamed: 0                    13 non-null     int64 
 1   transcript_id                 13 non-null     object
 2   petitioner_opening_text       13 non-null     object
 3   petitioner_full_text          13 non-null     object
 4   respondent_opening_statement  13 non-null     object
 5   respondent_full_text          13 non-null     object
dtypes: int64(1), object(5)
memory usage: 752.0+ bytes


In [97]:
def add_justice_questions(justice_name, opening_statement):
    messages = get_question_generation_prompt(justice_name, sample_os)
    response = get_model_response(messages)
    questions = parse_response(response)
    # return json string of list of questions to store in pandas df
    return json.dumps(questions)
    

In [101]:
# # test on subsample
# df_new = df.head(2).copy()
# df_new

# # add sotomayor, petitioner
# justice = 'Justice Sonia Sotomayor'
# df_new['questions_sotomayor_petitioner'] = df_new.apply(
#     lambda row: add_justice_questions(justice, row['petitioner_opening_text']), axis=1
# )
# df_new

,Unnamed: 0,transcript_id,petitioner_opening_text,petitioner_full_text,respondent_opening_statement,respondent_full_text,questions_sotomayor_petitioner
0,0,2024.23-621-t01,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Anthony A. Yang</speaker><text> Mr. C...,<speaker>Anthony A. Yang</speaker><text> Mr. C...,"[""Counselor, how do you respond to the argumen..."
1,1,2024.23-365 -t01,<speaker>Lisa S. Blatt</speaker><text> Thank y...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Easha Anand</speaker><text> Mr. Chief...,<speaker>Easha Anand</speaker><text> Mr. Chief...,"[""Counselor, can you explain how your proposed..."


In [108]:
# GENERATE FOR ALL
# add sotomayor, petitioner
justice = 'Justice Sonia Sotomayor'
df['questions_sotomayor_petitioner'] = df.apply(
    lambda row: add_justice_questions(justice, row['petitioner_opening_text']), axis=1
)
df.head()

,Unnamed: 0,transcript_id,petitioner_opening_text,petitioner_full_text,respondent_opening_statement,respondent_full_text,questions_sotomayor_petitioner
0,0,2024.23-621-t01,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Anthony A. Yang</speaker><text> Mr. C...,<speaker>Anthony A. Yang</speaker><text> Mr. C...,"[""Can you explain why you believe the Court sh..."
1,1,2024.23-365 -t01,<speaker>Lisa S. Blatt</speaker><text> Thank y...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Easha Anand</speaker><text> Mr. Chief...,<speaker>Easha Anand</speaker><text> Mr. Chief...,"[""Isn't it true that a preliminary injunction ..."
2,2,2024.23-852-t01,<speaker>Elizabeth B. Prelogar</speaker><text>...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Peter A. Patterson</speaker><text> Mr...,<speaker>Peter A. Patterson</speaker><text> Mr...,"[""Counselor, can you explain how the bright-li..."
3,3,2024.23-980-t01,<speaker>Kannon K. Shanmugam</speaker><text> T...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Kevin K. Russell</speaker><text> Mr. ...,<speaker>Kevin K. Russell</speaker><text> Mr. ...,"[""Counselor, doesn't your argument rely too he..."
4,4,2024.23-191 -t01,<speaker>Adam G. Unikowsky</speaker><text> Mr....,"<speaker>John G. Roberts, Jr.</speaker><text> ...","<speaker>Edmund G. LaCour, Jr.</speaker><text>...","<speaker>Edmund G. LaCour, Jr.</speaker><text>...","[""Counselor, doesn't the fact that a prelimina..."


In [ ]:
# add sotomayor, respondent
justice = 'Justice Sonia Sotomayor'
df['questions_sotomayor_respondent'] = df.apply(
    lambda row: add_justice_questions(justice, row['respondent_opening_statement']), axis=1
)
df.head()

In [111]:
df.head()

,Unnamed: 0,transcript_id,petitioner_opening_text,petitioner_full_text,respondent_opening_statement,respondent_full_text,questions_sotomayor_petitioner,questions_sotomayor_respondent
0,0,2024.23-621-t01,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Anthony A. Yang</speaker><text> Mr. C...,<speaker>Anthony A. Yang</speaker><text> Mr. C...,"[""Can you explain why you believe the Court sh...","[""Counselor, how do you respond to the argumen..."
1,1,2024.23-365 -t01,<speaker>Lisa S. Blatt</speaker><text> Thank y...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Easha Anand</speaker><text> Mr. Chief...,<speaker>Easha Anand</speaker><text> Mr. Chief...,"[""Isn't it true that a preliminary injunction ...","[""Counselor, how do you respond to the concern..."
2,2,2024.23-852-t01,<speaker>Elizabeth B. Prelogar</speaker><text>...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Peter A. Patterson</speaker><text> Mr...,<speaker>Peter A. Patterson</speaker><text> Mr...,"[""Counselor, can you explain how the bright-li...","[""How do you respond to the argument that a pr..."
3,3,2024.23-980-t01,<speaker>Kannon K. Shanmugam</speaker><text> T...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Kevin K. Russell</speaker><text> Mr. ...,<speaker>Kevin K. Russell</speaker><text> Mr. ...,"[""Counselor, doesn't your argument rely too he...","[""How do you respond to the argument that the ..."
4,4,2024.23-191 -t01,<speaker>Adam G. Unikowsky</speaker><text> Mr....,"<speaker>John G. Roberts, Jr.</speaker><text> ...","<speaker>Edmund G. LaCour, Jr.</speaker><text>...","<speaker>Edmund G. LaCour, Jr.</speaker><text>...","[""Counselor, doesn't the fact that a prelimina...","[""Counselor, isn't it true that your proposed ..."


In [112]:
# # save mid way in case notebook crashes
# out_fp = '2024_full_text_sotomayor_questions.csv'
# df.to_csv(out_fp, index=False)

In [113]:
# add alito, respondent
justice = 'Justice Samuel A. Alito'
df['questions_alito_respondent'] = df.apply(
    lambda row: add_justice_questions(justice, row['respondent_opening_statement']), axis=1
)
df.head()

,Unnamed: 0,transcript_id,petitioner_opening_text,petitioner_full_text,respondent_opening_statement,respondent_full_text,questions_sotomayor_petitioner,questions_sotomayor_respondent,questions_alito_respondent
0,0,2024.23-621-t01,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Anthony A. Yang</speaker><text> Mr. C...,<speaker>Anthony A. Yang</speaker><text> Mr. C...,"[""Can you explain why you believe the Court sh...","[""Counselor, how do you respond to the argumen...","[""Can you explain how your proposed bright-lin..."
1,1,2024.23-365 -t01,<speaker>Lisa S. Blatt</speaker><text> Thank y...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Easha Anand</speaker><text> Mr. Chief...,<speaker>Easha Anand</speaker><text> Mr. Chief...,"[""Isn't it true that a preliminary injunction ...","[""Counselor, how do you respond to the concern...","[""Counselor, how do you respond to the argumen..."
2,2,2024.23-852-t01,<speaker>Elizabeth B. Prelogar</speaker><text>...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Peter A. Patterson</speaker><text> Mr...,<speaker>Peter A. Patterson</speaker><text> Mr...,"[""Counselor, can you explain how the bright-li...","[""How do you respond to the argument that a pr...","[""Isn't your proposed bright-line rule overly ..."
3,3,2024.23-980-t01,<speaker>Kannon K. Shanmugam</speaker><text> T...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Kevin K. Russell</speaker><text> Mr. ...,<speaker>Kevin K. Russell</speaker><text> Mr. ...,"[""Counselor, doesn't your argument rely too he...","[""How do you respond to the argument that the ...","[""Can you explain why the legislative history ..."
4,4,2024.23-191 -t01,<speaker>Adam G. Unikowsky</speaker><text> Mr....,"<speaker>John G. Roberts, Jr.</speaker><text> ...","<speaker>Edmund G. LaCour, Jr.</speaker><text>...","<speaker>Edmund G. LaCour, Jr.</speaker><text>...","[""Counselor, doesn't the fact that a prelimina...","[""Counselor, isn't it true that your proposed ...","[""Counselor, how do you respond to the argumen..."


In [115]:
# GENERATE FOR ALL
# add alito, petitioner
justice = 'Justice Samuel A. Alito'
df['questions_alito_petitioner'] = df.apply(
    lambda row: add_justice_questions(justice, row['petitioner_opening_text']), axis=1
)
df.head()

,Unnamed: 0,transcript_id,petitioner_opening_text,petitioner_full_text,respondent_opening_statement,respondent_full_text,questions_sotomayor_petitioner,questions_sotomayor_respondent,questions_alito_respondent,questions_alito_petitioner
0,0,2024.23-621-t01,<speaker>Erika L. Maley</speaker><text> Mr. Ch...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Anthony A. Yang</speaker><text> Mr. C...,<speaker>Anthony A. Yang</speaker><text> Mr. C...,"[""Can you explain why you believe the Court sh...","[""Counselor, how do you respond to the argumen...","[""Can you explain how your proposed bright-lin...","[""How do you respond to the argument that the ..."
1,1,2024.23-365 -t01,<speaker>Lisa S. Blatt</speaker><text> Thank y...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Easha Anand</speaker><text> Mr. Chief...,<speaker>Easha Anand</speaker><text> Mr. Chief...,"[""Isn't it true that a preliminary injunction ...","[""Counselor, how do you respond to the concern...","[""Counselor, how do you respond to the argumen...","[""Isn't it true that some circuits have adopte..."
2,2,2024.23-852-t01,<speaker>Elizabeth B. Prelogar</speaker><text>...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Peter A. Patterson</speaker><text> Mr...,<speaker>Peter A. Patterson</speaker><text> Mr...,"[""Counselor, can you explain how the bright-li...","[""How do you respond to the argument that a pr...","[""Isn't your proposed bright-line rule overly ...","[""Can you clarify why you believe that a preli..."
3,3,2024.23-980-t01,<speaker>Kannon K. Shanmugam</speaker><text> T...,"<speaker>John G. Roberts, Jr.</speaker><text> ...",<speaker>Kevin K. Russell</speaker><text> Mr. ...,<speaker>Kevin K. Russell</speaker><text> Mr. ...,"[""Counselor, doesn't your argument rely too he...","[""How do you respond to the argument that the ...","[""Can you explain why the legislative history ...","[""How do you respond to the argument that a pr..."
4,4,2024.23-191 -t01,<speaker>Adam G. Unikowsky</speaker><text> Mr....,"<speaker>John G. Roberts, Jr.</speaker><text> ...","<speaker>Edmund G. LaCour, Jr.</speaker><text>...","<speaker>Edmund G. LaCour, Jr.</speaker><text>...","[""Counselor, doesn't the fact that a prelimina...","[""Counselor, isn't it true that your proposed ...","[""Counselor, how do you respond to the argumen...","[""Doesn't the fact that a preliminary injuncti..."


In [ ]:
out_fp = '../datasets/2024_full_text_sotomayor_alito_questions.csv'
df.to_csv(out_fp, index=False)

### Try sample call

In [80]:
sample_os = df['petitioner_opening_text'][0]
sample_os

'<speaker>Erika L. Maley</speaker><text> Mr. Chief Justice, and may it please the Court:  The prevailing party is the party who wins the lawsuit, obtaining a final judgment in its favor, or at least a party who obtains a ruling that the defendant is liable on the merits of one or more claims, such as a summary judgment or a judgment as a matter of law. A preliminary injunction is neither a final judgment nor a determination that the defendant is liable on the merits for violating federal law. It is simply a threshold prediction of the likelihood of success based on a truncated record and an initial, often hasty assessment of the law that may well prove to be faulty as the case proceeds. It provid es no enduring relief. By its nature, it is a temporary procedural order that dissolves upon final judgment. A preliminary injunction, therefore, does not make a plaintiff a prevailing party  within the meaning of that legal term of art, and, thus, no statutory exception to the de fault Americ

In [81]:
justice = 'Justice Sonia Sotomayor'

In [82]:
messages = get_question_generation_prompt(justice, sample_os)
messages

[{'role': 'system',
  'content': 'You are a legal expert trained to simulate the questioning style of Supreme Court justices during oral arguments. Below is an opening statement from a Supreme Court case and the name of a specific justice. Your task is to generate a list of questions that this justice is likely to ask in response to the opening statement. These questions should reflect the justice’s known priorities, jurisprudence, and typical questioning style, and they should be directly relevant to the arguments presented in the opening statement.\n\n        ### Instructions:\n        1. Carefully analyze the opening statement to identify key arguments, assumptions, or ambiguities.\n        2. Generate questions that the specified justice might ask.\n        3. Format each question with specific start and end tags for easier parsing:\n           - Use `<START_Q>` at the beginning of each question.\n           - Use `<END_Q>` at the end of each question.\n\n        ### Example:\n    

In [83]:
response = get_model_response(messages)

In [84]:
content = response.content.decode('utf-8')
content

'{"id":"chat-daff385ee8ef45ae8f5659e1844e80a3","object":"chat.completion","created":1732721203,"model":"meta-llama/Meta-Llama-3.1-70B-Instruct","choices":[{"index":0,"message":{"role":"assistant","content":"A list of questions that Justice Sonia Sotomayor is likely to ask:\\n<START_Q> Counselor, doesn\'t the temporary nature of a preliminary injunction actually underscore its importance in protecting the rights of plaintiffs who may not have the resources to see the case through to a final judgment, and wouldn\'t denying them prevailing party status under these circumstances undermine the very purpose of Section 1988? <END_Q>\\n<START_Q> You rely on legal dictionaries from the time of Section 1988\'s enactment, but isn\'t it also relevant to consider the legislative history and the intent behind the statute, which was to encourage the enforcement of civil rights laws and provide a means for plaintiffs to recover attorneys\' fees in doing so? <END_Q>\\n<START_Q> How do you respond to th

In [91]:
questions = parse_response(response)
questions

["Counselor, doesn't the temporary nature of a preliminary injunction actually underscore its importance in protecting the rights of plaintiffs who may not have the resources to see the case through to a final judgment, and wouldn't denying them prevailing party status under these circumstances undermine the very purpose of Section 1988?",
 "You rely on legal dictionaries from the time of Section 1988's enactment, but isn't it also relevant to consider the legislative history and the intent behind the statute, which was to encourage the enforcement of civil rights laws and provide a means for plaintiffs to recover attorneys' fees in doing so?",
 "How do you respond to the argument that adopting a bright-line rule that a preliminary injunction does not make a plaintiff a prevailing party could have a chilling effect on plaintiffs who seek to vindicate their rights through preliminary relief, particularly in cases where the defendant's actions are causing irreparable harm?",
 "Doesn't th

In [117]:
type(questions)
for q in questions:
    print(q)

Counselor, doesn't the temporary nature of a preliminary injunction actually underscore its importance in protecting the rights of plaintiffs who may not have the resources to see the case through to a final judgment, and wouldn't denying them prevailing party status under these circumstances undermine the very purpose of Section 1988?
You rely on legal dictionaries from the time of Section 1988's enactment, but isn't it also relevant to consider the legislative history and the intent behind the statute, which was to encourage the enforcement of civil rights laws and provide a means for plaintiffs to recover attorneys' fees in doing so?
How do you respond to the argument that adopting a bright-line rule that a preliminary injunction does not make a plaintiff a prevailing party could have a chilling effect on plaintiffs who seek to vindicate their rights through preliminary relief, particularly in cases where the defendant's actions are causing irreparable harm?
Doesn't the Court's prec

In [118]:
qs_str = json.dumps(questions)
qs_str

'["Counselor, doesn\'t the temporary nature of a preliminary injunction actually underscore its importance in protecting the rights of plaintiffs who may not have the resources to see the case through to a final judgment, and wouldn\'t denying them prevailing party status under these circumstances undermine the very purpose of Section 1988?", "You rely on legal dictionaries from the time of Section 1988\'s enactment, but isn\'t it also relevant to consider the legislative history and the intent behind the statute, which was to encourage the enforcement of civil rights laws and provide a means for plaintiffs to recover attorneys\' fees in doing so?", "How do you respond to the argument that adopting a bright-line rule that a preliminary injunction does not make a plaintiff a prevailing party could have a chilling effect on plaintiffs who seek to vindicate their rights through preliminary relief, particularly in cases where the defendant\'s actions are causing irreparable harm?", "Doesn\

In [119]:
qs_orig = json.loads(qs_str)
qs_orig

["Counselor, doesn't the temporary nature of a preliminary injunction actually underscore its importance in protecting the rights of plaintiffs who may not have the resources to see the case through to a final judgment, and wouldn't denying them prevailing party status under these circumstances undermine the very purpose of Section 1988?",
 "You rely on legal dictionaries from the time of Section 1988's enactment, but isn't it also relevant to consider the legislative history and the intent behind the statute, which was to encourage the enforcement of civil rights laws and provide a means for plaintiffs to recover attorneys' fees in doing so?",
 "How do you respond to the argument that adopting a bright-line rule that a preliminary injunction does not make a plaintiff a prevailing party could have a chilling effect on plaintiffs who seek to vindicate their rights through preliminary relief, particularly in cases where the defendant's actions are causing irreparable harm?",
 "Doesn't th

In [120]:
type(questions)
for q in questions:
    print(q)

Counselor, doesn't the temporary nature of a preliminary injunction actually underscore its importance in protecting the rights of plaintiffs who may not have the resources to see the case through to a final judgment, and wouldn't denying them prevailing party status under these circumstances undermine the very purpose of Section 1988?
You rely on legal dictionaries from the time of Section 1988's enactment, but isn't it also relevant to consider the legislative history and the intent behind the statute, which was to encourage the enforcement of civil rights laws and provide a means for plaintiffs to recover attorneys' fees in doing so?
How do you respond to the argument that adopting a bright-line rule that a preliminary injunction does not make a plaintiff a prevailing party could have a chilling effect on plaintiffs who seek to vindicate their rights through preliminary relief, particularly in cases where the defendant's actions are causing irreparable harm?
Doesn't the Court's prec